In [1]:
from datetime import datetime
from urllib.request import Request, urlopen
from urllib.error import HTTPError, URLError
from re import sub
from lxml import html
import requests
from requests.exceptions import ConnectionError
import csv
import json
import re
import time
import random
from random import randint
import ast
import pandas as pd
import warnings
warnings.simplefilter(action='ignore')
from pprint import pprint

In [2]:
user_agent_dict = {
"Mozilla/5.0 (Linux; Android 13; SM-G990B Build/TP1A.220624.014) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/113.0.5672.162 Mobile Safari/537.36 GNews Android/2022131842":1,
"Mozilla/5.0 (Linux; Android 10; M2006C3MNG Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/113.0.5672.163 Mobile Safari/537.36":2,
"Mozilla/5.0 (Linux; Android 10; SM-J600FN Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/113.0.5672.162 Mobile Safari/537.36 [FB_IAB/FB4A;FBAV/417.0.0.33.65;] ":3,
"Mozilla/5.0 (Linux; Android 11; SM-A805F Build/RP1A.200720.012; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/113.0.5672.132 Mobile Safari/537.36 [FB_IAB/FB4A;FBAV/416.0.0.35.85;]":4,
"Mozilla/5.0 (Linux; arm_64; Android 11; Mi Note 10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5475.114 Mobile Safari/537.36":5,
"Mozilla/5.0 (Linux; Android 13; CPH2145 Build/TP1A.220905.001; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/114.0.5735.58 Mobile Safari/537.36 [FB_IAB/FB4A;FBAV/416.0.0.35.85;]Mozilla/5.0 (Linux; Android 6.0.1; LG-K220 Build/MXB48T; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/86.0.4240.99 Mobile Safari/537.36 GSA/11.31.12.21.arm":6
            }

In [3]:
def clean(text):
    if text:
        return ' '.join(' '.join(text).split())
    return None

In [4]:
def rand_agent():
    user_agent = [x for x in user_agent_dict.keys()]
    return random.choice(user_agent)

In [5]:
def get_headers():
    # Creating headers.
    headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
               'accept-encoding': 'gzip, deflate, sdch, br',
               'accept-language': 'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
               'cache-control': 'max-age=0',
               'upgrade-insecure-requests': '1',
               'user-agent' : 'Mozilla/5.0 (Linux; Android 6.0; E5633 Build/30.2.B.1.21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.81 Mobile Safari/537.36'
               }
    return headers

In [8]:
def create_url(state, pricetype):
    # Creating Zillow URL based on the filter.
    if pricetype == 'sold':
        # for sale
        url = 'https://urldefense.com/v3/__https://www.zillow.com/*7B0*7D/sold/'.format(state)*5Cn__;JSUl!!DZ3fjg!_DXleXNXcajNqNdK5TWQMZ1NIXSScDQfSZuS72R4H-27SkBpV0zC_LX2mttTnpPfxaOZdk592PH9ncknRm1WHW7eLA$     
    elif pricetype == "for sale":
        url = 'https://urldefense.com/v3/__https://www.zillow.com/homes/*7B0*7D/'.format(state)*5Cn__;JSUl!!DZ3fjg!_DXleXNXcajNqNdK5TWQMZ1NIXSScDQfSZuS72R4H-27SkBpV0zC_LX2mttTnpPfxaOZdk592PH9ncknRm2tQC3G4w$     
    else:
        # for rent
        url = 'https://urldefense.com/v3/__https://www.zillow.com/*7B0*7D/rentals/'.format(state)*5Cn__;JSUl!!DZ3fjg!_DXleXNXcajNqNdK5TWQMZ1NIXSScDQfSZuS72R4H-27SkBpV0zC_LX2mttTnpPfxaOZdk592PH9ncknRm3kYd2w9A$     
    print(url)
    return url

SyntaxError: invalid syntax (2160039490.py, line 5)

In [9]:
def save_to_file(response):
    # saving response to `response.html`
    with open("response.html", 'w', encoding="utf-8") as fp:
        fp.write(response.text)


In [29]:
def write_data_to_csv(data, state, pricetype):
    # saving scraped data to csv.

    with open("C:/Users/laisi/Dropbox/00-Career/2023_9_GardenStar Group/Data and Codes/%s-price-%s-%s.csv" 
              % (pricetype, state, datetime.today().strftime('%Y-%m-%d')), 'w', newline='') as csvfile:

        fieldnames = ['zpid', 'type', 'title', 'address', 'city', 'state', 'zipcode', 
                      'facts and features', 'price', 'bedrooms', 'bathrooms', 'sqft',
                      'lot_size', 'lot_area_unit', 'year_built', 'price_history', 'tax_history', 
                      'latlong', 'property_url', 'photo_main', 'is_published', 'list_date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [27]:
def write_historical_data_to_csv(data, state, price_or_tax, pricetype):
    # saving scraped data to csv.

    with open("C:/Users/laisi/Dropbox/00-Career/2023_9_GardenStar Group/Data and Codes/%s-%s-history-%s-%s.csv" % (pricetype, price_or_tax, state, datetime.today().strftime('%Y-%m-%d')), 'w', newline='') as csvfile:

        if price_or_tax == 'price':
            fieldnames = ['zpid', 'date', 'event', 'price']
        else:
            fieldnames = ['zpid', 'date', 'taxPaid']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [12]:
def get_response(url):
    # Getting response from zillow.com.

    for i in range(5):
        response = requests.get(url, headers=get_headers())
        print("status code received:", response.status_code)
        if response.status_code != 200:
            # saving response to file for debugging purpose.
            save_to_file(response)
            continue
        else:
            save_to_file(response)
            return response
    return None

In [13]:
def find_valid_parentheses_index(text):
    # find the index of '}' that make the text start with '{' a valid parenthesis
    # input: a text start with '{'
    count = 0
    for i in range(len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
        else:
            continue
        if count == 0:
            return i

In [14]:
def get_price_history(home_detail_data):
    # input: a long string contains price history
    # return Type: a list consis of dictionaries as records of price history
    try:
        startpattern = '\\"priceHistory\\":'
        endpattern = ']'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = home_detail_data[start:].find(endpattern) + start + 1
        pricehiststr = home_detail_data[start:end]
        result = re.sub(r"\\", "", pricehiststr, flags=re.I)
        resultjson = json.loads(result)
        #keylist = ['time','event','price','source']
        keylist = ['time','event','price']
        price_history = []
        for history in resultjson:
            record = {x : history[x] if x!='time' 
                      else datetime.fromtimestamp(history[x] / 1e3).strftime('%Y-%m-%d') for x in keylist}
            price_history.append(record)
        return price_history
    except:
        print("Can't find price history.")
        return None
#        return [{'time': '','price': None,'source': '','event': ''}]

In [15]:
def get_tax_history(home_detail_data):
    # input: a long string contains tax history
    # return Type: a list consis of dictionaries as records of tax history
    try:
        startpattern = '\\"taxHistory\\":'
        endpattern = ']'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = home_detail_data[start:].find(endpattern) + start + 1
        taxhiststr = home_detail_data[start:end]
        result = re.sub(r"\\", "", taxhiststr, flags=re.I)
        resultjson = json.loads(result)
        keylist = ['time','taxPaid']
        tax_history = []
        for history in resultjson:
            record = {x : history[x] if x!='time' 
                      else datetime.fromtimestamp(history[x] / 1e3).strftime('%Y') for x in keylist}
            tax_history.append(record)
        return tax_history
    except:
        print("Can't find tax history.")
        return None

In [16]:
def get_home_facts_and_features(home_detail_data):
    # input: a long string contains facts and features
    # return Type: a dictionary of facts and features
    try:
        startpattern = '{\\"property\\":'
        endpattern = ']'
        start = home_detail_data.find(startpattern) + len(startpattern)
        end = find_valid_parentheses_index(home_detail_data[start:]) + start + 1
        featuresstr = home_detail_data[start:end]
        result = re.sub(r"\\", "", featuresstr, flags=re.I)
        features = ['yearBuilt','lotAreaUnit']
        featuresjson = {}
        startindx_year=result.find('yearBuilt')
        i = 1
        while result[startindx_year+i]!=",":
            i=i+1
            yearstr=result[startindx_year:startindx_year+i]
            numeric_filter = filter(str.isdigit, yearstr)
            numeric_string = "".join(numeric_filter)
            featuresjson['yearBuilt']=numeric_string
    
        startindx_lot=result.find('lotAreaUnit')
        while result[startindx_lot+i]!=",":
            i=i+1
            lotstr=result[startindx_lot:startindx_lot+i]
            unitstr=lotstr.split(":")[-1]
            letter_filter = filter(str.isalpha, unitstr)
            letter_string = "".join(letter_filter)
            featuresjson['lotAreaUnit']=letter_string
        return featuresjson
    except:
        print("Can't get home details.")
        return {'yearBuilt': None,'lotAreaUnit': ''}

In [17]:
def get_homedetails(property_url):
    # input: url
    # return Type: dictionary
    for i in range(5):
        random_agent = {'User-Agent': rand_agent()}
        #print("Send request...")
        req = Request(property_url, headers=random_agent)
        try:
            #print("Open webpage...")
            webpage = urlopen(req).read()
            #print("Get webpage..")
        except ConnectionError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        except ConnectionResetError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        except URLError as exception:
            print('Try to get home details response ({}) time.'.format(i+1))
            print(exception)
            time.sleep(randint(1,5))
            continue
        parser = html.fromstring(webpage)
        try:
            raw_json_data = parser.get_element_by_id('__NEXT_DATA__').text
            agentidx = user_agent_dict[random_agent.get('User-Agent')]
            return raw_json_data, agentidx
        except:
            print('Try to get home details response ({}) time.'.format(i+1))
            try:
                response = get_response(property_url)
                parser = html.fromstring(response.text)
                raw_json_data = parser.get_element_by_id('__NEXT_DATA__').text
                agentidx = user_agent_dict[random_agent.get('User-Agent')]
                return raw_json_data, agentidx
            except:
                time.sleep(randint(3,10))
                continue
    print("Can't find home details.")
    return None, None

In [18]:
def get_data_from_json(raw_json_data):
    
    cleaned_data = clean(raw_json_data)
    properties_list = []
    try:
        json_data = json.loads(cleaned_data)
        search_results = json_data.get('props').get('pageProps').get('searchPageState').get('cat1').get('searchResults').get('listResults',[])
        print(json_data.get('props').get('pageProps').get('searchPageState').get('searchPageSeoObject'))
        total_listing = int(json_data.get('props').get('pageProps').get('searchPageState').get('cat1').get('searchList').get('totalResultCount'))
        count = 0
        result_size = len(search_results)
        print("how many results are we processing?", result_size)

        valid_agent_lst = [0 for _ in range(len(user_agent_dict))]
        
        for properties in search_results:
            #print("processing result", count, properties)
            print('({}) item of the page'.format(count+1))
            zpid = properties.get('zpid')
            statusText = properties.get('statusText')
            if not zpid.isdigit() or statusText == 'Off market':
                print("bypassing zillow id:", zpid, "statusText:", statusText)
                continue
            address = properties.get('address')
            property_info = properties.get('hdpData', {}).get('homeInfo')
            city = property_info.get('city')
            state = property_info.get('state')
            zipcode = property_info.get('zipcode')
            lot_size = property_info.get('lotAreaValue')
            hometype = property_info.get('homeType')
            try:
                price = int(property_info.get('price'))
            except:
                price = 0
            bedrooms = properties.get('beds')
            bathrooms = properties.get('baths')
            area = properties.get('area')
            property_url = properties.get('detailUrl')
            info = f'{bedrooms} bds,{bathrooms} ba,{area} sqft,{property_url}'
            title = properties.get('statusText')
            photo_main = properties.get('imgSrc')
            latLong = properties.get('latLong')
            #print("Get home details..")
            homedetails,agentidx = get_homedetails(property_url)
            if agentidx:
                valid_agent_lst[agentidx-1] += 1
            home_features = get_home_facts_and_features(homedetails)
            #if home_features.get('price'):
            #   price = home_features.get('price')
            year_built = home_features.get('yearBuilt')
            lot_area_unit = home_features.get('lotAreaUnit')
            price_history = get_price_history(homedetails)
            tax_history = get_tax_history(homedetails)


            data = {'address': address,
                    'city': city,
                    'state': state,
                    'zpid' : zpid,
                    'zipcode': zipcode,
                    'price': price,
                    'bedrooms': bedrooms,
                    'bathrooms': bathrooms,
                    'sqft': area,
                    'year_built': year_built,
                    'lot_size': lot_size,
                    'lot_area_unit': lot_area_unit,
                    'latlong': latLong,
                    'price_history': price_history,
                    'tax_history': tax_history,
                    'facts and features': info,
                    'title': title,
                    'type': hometype,
                    'property_url': property_url,
                    'photo_main': photo_main,
                    'is_published': True, #set this as false for now
                    'list_date': datetime.today().strftime('%Y-%m-%d')}
            properties_list.append(data)
            count += 1
        return total_listing, properties_list, valid_agent_lst

    except ValueError:
        print("Invalid json", ValueError)
        return None


In [19]:
def remove_duplicates(scraped_data):
    zpid_list = []
    cleaned_scraped_data = []
    for data in scraped_data:
        if data['zpid'] not in zpid_list:
            zpid_list.append(data['zpid'])
            cleaned_scraped_data.append(data)
    return cleaned_scraped_data

In [20]:
def get_price_tax_history_list(cleaned_scraped_data):
    
    # initializing the fields and price tax list 
    price_history_list = []
    tax_history_list = []
      
    # extracting hitorical data for each house
    for row in cleaned_scraped_data: 
        zpid = row['zpid']
        if not row['price_history']:
            price_history_single = [{'zpid':zpid, 'date':'', 'event':'', 'price':None}]
        elif row['price_history'] == '[]':
            price_history_single = [{'zpid':zpid, 'date':'', 'event':'', 'price':None}]
        else:
            price = row['price_history']
            if type(price) is not list:
                price = [price]
            price_history_single = [{'zpid':zpid, 'date':d['time'], 'event':d['event'], 'price':d['price']} for d in price]

        if not row['tax_history']:
            tax_history_single = [{'zpid':zpid, 'date':'', 'taxPaid':None}]
        elif row['tax_history'] == '[]':
            tax_history_single = [{'zpid':zpid, 'date':'', 'taxPaid':None}]
        else:
            tax = row['tax_history']
            if type(tax) is not list:
                tax = [tax]
            tax_history_single = [{'zpid':zpid, 'date':d['time'], 'taxPaid':d['taxPaid']} for d in tax]
            
        price_history_list.extend(price_history_single)
        tax_history_list.extend(tax_history_single)
            
    return price_history_list, tax_history_list


In [22]:
pricetype = 'for rent'
state = 'flowery-branch-ga'
scraped_data = []

In [24]:
######################################################################################################################
# change url for each page
for page in range(1, 7):
    url = f'https://www.zillow.com/flowery-branch-ga/sold/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A{page}%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-84.14669534570314%2C%22east%22%3A-83.74500772363282%2C%22south%22%3A34.07713392267184%2C%22north%22%3A34.30318865683498%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A8713%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A12%7D'     
    for i in range(100):
        response = get_response(url)
        random_agent = {'User-Agent': rand_agent()}
        req = Request(url, headers=random_agent)
        webpage = urlopen(req).read()
        parser = html.fromstring(webpage)
        xpath_expression = '//script[@id="__NEXT_DATA__"]//text()'
        raw_json_data = parser.xpath(xpath_expression)
        if not raw_json_data:
            time.sleep(randint(3,10)) # break down the time to prevent anti data crawling
            continue
        total_listing, prop_list, valid_agent_lst = get_data_from_json(raw_json_data)
        break
    scraped_data.extend(prop_list)

status code received: 200
{'baseUrl': '/flowery-branch-ga/sold/', 'windowTitle': 'Recently Sold Homes in Flowery Branch GA - 3774 Transactions | Zillow', 'metaDescription': 'Browse data on the 3774 recent real estate transactions in Flowery Branch GA. Great for discovering comps, sales history, photos, and more.', 'robotsTagContent': 'NOINDEX, NOFOLLOW'}
how many results are we processing? 41
(1) item of the page
(2) item of the page
(3) item of the page
(4) item of the page
(5) item of the page
(6) item of the page
(7) item of the page
(8) item of the page
(9) item of the page
(10) item of the page
(11) item of the page
(12) item of the page
(13) item of the page
(14) item of the page
(15) item of the page
(16) item of the page
(17) item of the page
(18) item of the page
(19) item of the page
(20) item of the page
(21) item of the page
(22) item of the page
(23) item of the page
(24) item of the page
(25) item of the page
(26) item of the page
(27) item of the page
(28) item of the pa

In [25]:
# Check the length of dataset
len(scraped_data) 

246

In [30]:
#######################################################################################################################
# create a folder called zillow_historical_data_zipcode before run the rest part
# write data out to a csv file
cleaned_scraped_data = remove_duplicates(scraped_data)
write_data_to_csv(cleaned_scraped_data, state, pricetype)

price_history_list, tax_history_list = get_price_tax_history_list(cleaned_scraped_data)
write_historical_data_to_csv(price_history_list, state, 'price', pricetype)
write_historical_data_to_csv(tax_history_list, state, 'tax', pricetype)